In [2]:
"""
-word count: 
"""
# nombre de partition (cloisons) par default dans le SC = Le nombre de Workers
from time import time
from pyspark import SparkContext
# spark context avec 4 workers
sc = SparkContext(master="local[4]")

In [5]:
%%time
#lecture fichier texte :
datadir = 'mobydick'
filename = 'mobydick.txt'
textFile = sc.textFile(datadir+'/'+filename)
type(textFile)

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 160 ms


In [20]:
%%time
# on definit les plan d'execution :
keyWords = textFile.flatMap(lambda phrase : phrase.split(" ")).filter(lambda word : word!='').map(lambda word : (word,1)).reduceByKey(lambda x,y : x+y)
print(keyWords.sample(False,0.001).collect())

[('themselves', 20), ('here,', 23), ('descending', 4), ('proper', 12), ('ladder,', 4), ('reasons,', 3), ('run,', 1), ('good.', 3), ('descend', 2), ('succeed,', 1), ('widely', 2), ('lessness', 1), ('Tormentoto,', 1), ('repre-', 1), ('MAST-HEAD', 5), ('Eaimond', 1), ('gale,', 6), ('lubber-like', 1), ('trump', 2), ('untraditionally', 1), ('shoulder,', 1), ('stars.', 1), ('likened', 1), ('crunching', 1), ('referred', 3), ('circular.', 1), ('Procopius', 1), ('back-foremost', 1), ('midway', 1), ('things),', 1)]
CPU times: user 50 ms, sys: 10 ms, total: 60 ms
Wall time: 1.32 s


In [21]:
# affichage du plan
print('Plan : ',keyWords.toDebugString().decode())

Plan :  (2) PythonRDD[39] at RDD at PythonRDD.scala:48 []
 |  MapPartitionsRDD[37] at mapPartitions at PythonRDD.scala:122 []
 |  ShuffledRDD[36] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[35] at reduceByKey at <timed exec>:2 []
    |  PythonRDD[34] at reduceByKey at <timed exec>:2 []
    |  mobydick/mobydick.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0 []
    |  mobydick/mobydick.txt HadoopRDD[2] at textFile at NativeMethodAccessorImpl.java:0 []


In [23]:
%%time
# comptage des mots :
countWords = keyWords.count()
# on ne prend que les nombres par les mots pour sommer
sumWords = keyWords.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Distinct words count : ',countWords,'\nWords sum : ',sumWords)

Distinct words count :  19840 
Words sum :  115314
CPU times: user 20 ms, sys: 40 ms, total: 60 ms
Wall time: 490 ms


In [27]:
%%time
# 5 mots les plus utilisés :
#methode python 
colFile = keyWords.collect()
colFile.sort(key=lambda x:x[1])
print('Top 5 most used words in moby dick : \n'+'\n'.join(['%s:\t%d'% colFile for colFile in reversed(colFile[-5:])]))

Top 5 most used words in moby dick : 
the:	6611
of:	3460
and:	2969
a:	2466
to:	2339
CPU times: user 50 ms, sys: 0 ns, total: 50 ms
Wall time: 220 ms


In [31]:
%%time
# methode Spark 
keyWords = textFile.flatMap(lambda phrase : phrase.split(" "))\
    .filter(lambda word : word!='')\
    .map(lambda word : (word,1))

BkeyWords = keyWords.reduceByKey(lambda x,y:x+y)

CPU times: user 30 ms, sys: 10 ms, total: 40 ms
Wall time: 81.2 ms


In [37]:
%%time
# reverse key and sort
reverseWords = BkeyWords.map(lambda x:(x[1],x[0]))
sortedWords = reverseWords.sortByKey(ascending = False)

CPU times: user 60 ms, sys: 20 ms, total: 80 ms
Wall time: 640 ms


In [38]:
# plan du spark methode
print('Plan Spark method : ',sortedWords.toDebugString().decode())

Plan Spark method :  (2) PythonRDD[80] at RDD at PythonRDD.scala:48 []
 |  MapPartitionsRDD[78] at mapPartitions at PythonRDD.scala:122 []
 |  ShuffledRDD[77] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[76] at sortByKey at <timed exec>:3 []
    |  PythonRDD[75] at sortByKey at <timed exec>:3 []
    |  MapPartitionsRDD[53] at mapPartitions at PythonRDD.scala:122 []
    |  ShuffledRDD[52] at partitionBy at NativeMethodAccessorImpl.java:0 []
    +-(2) PairwiseRDD[51] at reduceByKey at <timed exec>:4 []
       |  PythonRDD[50] at reduceByKey at <timed exec>:4 []
       |  mobydick/mobydick.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0 []
       |  mobydick/mobydick.txt HadoopRDD[2] at textFile at NativeMethodAccessorImpl.java:0 []


In [42]:
%%time
# top 5 by spark method
colFile = sortedWords.take(5)
print('Top 5 most used words in moby dick : \n'+'\n'.join(['%d:\t%s'%c for c in colFile]))

Top 5 most used words in moby dick : 
6611:	the
3460:	of
2969:	and
2466:	a
2339:	to
CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 185 ms
